### This script file is used to analysis Default.csv dataset.

<b>We want to use logistic regression to predict the probability of default using income and balance on the Default data set. Our goal is to estimate the test error of this logistic regression model using the validation set approach. Do not forget to set a random seed before beginning your analysis.</b>

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix, accuracy_score

/Users/ruotwang/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
df = pd.read_csv('data/Default.csv')
df = df.drop('index', axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
default    10000 non-null object
student    10000 non-null object
balance    10000 non-null float64
income     10000 non-null float64
dtypes: float64(2), object(2)
memory usage: 312.6+ KB


In [3]:
description = df.describe()
print(description)

            balance        income
count  10000.000000  10000.000000
mean     835.374886  33516.981876
std      483.714985  13336.639563
min        0.000000    771.967729
25%      481.731105  21340.462905
50%      823.636973  34552.644800
75%     1166.308387  43807.729275
max     2654.322576  73554.233500


<b>(a) Fit a logistic regression model that uses income and balance to predict default.</b>

In [4]:
# Convert 'default' column into binary representation
df['default_bi'] = np.zeros(len(df.index))
df.loc[df['default'] == 'Yes', 'default_bi'] = 1

In [5]:
preds = ['income', 'balance']
lr_form = 'default_bi~' + '+'.join(preds)
log_reg = smf.glm(formula=lr_form, data=df, family=sm.families.Binomial()).fit()
log_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:             default_bi   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9997
Model Family:                Binomial   Df Model:                            2
Link Function:                  logit   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -789.48
Date:                Wed, 01 Nov 2017   Deviance:                       1579.0
Time:                        17:57:56   Pearson chi2:                 6.95e+03
No. Iterations:                     9                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -11.5405      0.435    -26.544      0.000     -12.393     -10.688
income      2.081e-05   4.99e-06      4.174      0.000     1.1e-05    3.06e-05
balance        0.0056      0.000     24.835      0.000       0.005       0.006
==============================================================================
"""

<b>(b) Using the validation set approach, estimate the test error of this model. In order to do this, you must perform the following steps:
1. Split the sample set into a training set and a validation set.
1. Fit a multiple logistic regression model using only the training observations.
1. Obtain a prediction of default status for each individual in the validation set by computing the posterior probability of default for that individual, and classifying the individual to the default category if the posterior probability is greater than 0.5.
1. Compute the validation set error, which is the fraction of the observations in the validation set that are misclassified.</b>

In [6]:
# prepare data, choose 20% data as test data
preds = ['income', 'balance']
X_train, X_test, Y_train, Y_test = train_test_split(df[preds], df['default_bi'], test_size=0.2)

In [7]:
# Initiate logistic regression object
logit_clf = LogisticRegression()
res_logit_clf = logit_clf.fit(X_train, Y_train)
Y_pred = res_logit_clf.predict(X_test)

confusion_matrix(Y_test, Y_pred)

array([[1943,    0],
       [  57,    0]])

In [8]:
print("Validation set error:", 1 - accuracy_score(Y_test, Y_pred))

Validation set error: 0.0285


<b>(c) Repeat the process in (b) three times, using three different splits of the observations into a training set and a validation set. Describe your findings and comment on the results obtained.</b>

In [9]:
# Calculated mean error on validation sets
def get_cv_err(x_data, y_data, cvobj, regobj):

    cv_errs = []
    for train_idx, test_idx in cvobj.split(x_data):

        xtrain, xtest = x_data[train_idx], x_data[test_idx]
        ytrain, ytest = y_data[train_idx], y_data[test_idx]

        res_reg = regobj.fit(xtrain, ytrain)
        pred_reg = res_reg.predict(xtest)

        # Reshape necessary because predition produces a (1, n) numpy array, while ytest is (n, 1)
        cv_errs.append(1 - accuracy_score(ytest, pred_reg))
    
    return cv_errs


# K-fold CV strategy
def kfold_err(x_data, y_data, num_splits=10):
    
    # Kfold Cross-validation
    kfcv = KFold(n_splits=num_splits)

    klreg = LogisticRegression()

    return get_cv_err(x_data, y_data, kfcv, klreg)

In [10]:
nsplits = 3
print('Validation set errors for ' + str(nsplits) + ' splits')
print(kfold_err(np.array(df[preds]), np.array(df['default_bi']), num_splits=nsplits))

Validation set errors for 3 splits
[0.035392921415716816, 0.032703270327032685, 0.033003300330032959]


In [11]:
nsplits = 10
print('Validation set errors for ' + str(nsplits) + ' splits')
print(kfold_err(np.array(df[preds]), np.array(df['default_bi']), num_splits=nsplits))

Validation set errors for 10 splits
[0.033000000000000029, 0.037000000000000033, 0.031000000000000028, 0.037000000000000033, 0.037000000000000033, 0.024000000000000021, 0.042000000000000037, 0.027000000000000024, 0.031000000000000028, 0.037000000000000033]


In [12]:
nsplits = 30
print('Validation set errors for ' + str(nsplits) + ' splits')
print(kfold_err(np.array(df[preds]), np.array(df['default_bi']), num_splits=nsplits))

Validation set errors for 30 splits
[0.0239520958083832, 0.041916167664670656, 0.032934131736526928, 0.032934131736526928, 0.041916167664670656, 0.038922155688622784, 0.047904191616766512, 0.0239520958083832, 0.017964071856287456, 0.050898203592814384, 0.033033033033033066, 0.030030030030030019, 0.039039039039039047, 0.036036036036036001, 0.036036036036036001, 0.027027027027026973, 0.027027027027026973, 0.018018018018018056, 0.042042042042042094, 0.042042042042042094, 0.042042042042042094, 0.024024024024024038, 0.030030030030030019, 0.027027027027026973, 0.021021021021020991, 0.045045045045045029, 0.027027027027026973, 0.033033033033033066, 0.042042042042042094, 0.036036036036036001]


We can see as we increase the split set number, the variance in each validation set also increases.

<b>(d) Now consider a logistic regression model that predicts the probability of default using income, balance, and a dummy variable for student. Estimate the test error for this model using the validation set approach. Comment on whether or not including a dummy variable for student leads to a reduction in the test error rate.</b>

In [13]:
# Convert 'student' column into binary representation
df['student_bi'] = np.zeros(len(df.index))
df.loc[df['student'] == 'Yes', 'student_bi'] = 1

In [14]:
preds_new = ['income', 'balance', 'student_bi']
lr_form_new = 'default_bi~' + '+'.join(preds_new)
logreg_new = smf.glm(formula=lr_form_new, data=df, family=sm.families.Binomial()).fit()

logreg_new.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:             default_bi   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9996
Model Family:                Binomial   Df Model:                            3
Link Function:                  logit   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -785.77
Date:                Wed, 01 Nov 2017   Deviance:                       1571.5
Time:                        17:58:10   Pearson chi2:                 7.00e+03
No. Iterations:                     9                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -10.8690      0.492    -22.079      0.000     -11.834      -9.904
income      3.033e-06    8.2e-06      0.370      0.712    -1.3e-05    1.91e-05
balance        0.0057      0.000     24.737      0.000       0.005       0.006
student_bi    -0.6468      0.236     -2.738      0.006      -1.110      -0.184
==============================================================================
"""

In [15]:
print('Validation set errors for 3 splits')
print(kfold_err(np.array(df[preds_new]), np.array(df['default_bi']), num_splits=3))

Validation set errors for 3 splits
[0.035392921415716816, 0.032703270327032685, 0.033003300330032959]


It doesn’t seem that adding the “student” dummy variable leads to a reduction in the validation set estimate of the test error rate.